In [36]:
from backend.dsm2_reader import read_scenario_dir
import pandas as pd
import altair as alt


In [37]:
data = read_scenario_dir("C:/Users/Inigo/Projects/sdg-dashboard/data", v7_filter="_7")

mathing:		C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV1Ma_hydro_7.dss <----> C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV1Ma_SDG_7.dss
DSS file opened successfully.
DSS file opened successfully.


In [ ]:
elev_list =['MID_GATE_UP','MID_GATE_DOWN','GLC_GATE_UP','GLC_GATE_DOWN','OLD_GATE_UP','OLD_GATE_DOWN']
flow_list = ['GLC_FLOW_FISH','MID_FLOW_FISH','MID_FLOW_GATE','OLD_FLOW_FISH','OLD_FLOW_GATE']
stn_name = ['MHO','DGL','OLD']

sdg = data['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss']['sdg']
sdg_stage = sdg[sdg['gate_op'].isin(elev_list)]
# print(sdg_stage['gate_op'].unique())

sdg_flow = sdg[sdg['gate_op'].isin(flow_list)]

hydro = data['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss']['hydro']
hydro_gateop = hydro[hydro['gate'].isin(stn_name)]
hydro_gateop.rename(columns={'MID_GATEOP':'MHO','GLC_GATEOP':'DGL','OLD_GATEOP':'OLD'}, inplace=True) # renaming to make it easier later.
# print(hydro_gateop['gate'].unique())

# print(hydro_gateop.head())


['GLC_GATE_DOWN' 'OLD_GATE_DOWN' 'GLC_GATE_UP' 'MID_GATE_UP' 'OLD_GATE_UP'
 'MID_GATE_DOWN']
             datetime  value unit        gate_op scenario
0 2015-12-31 00:15:00    NaN  CFS  OLD_FLOW_GATE   FPV1MA
1 2015-12-31 00:30:00    NaN  CFS  OLD_FLOW_GATE   FPV1MA
2 2015-12-31 00:45:00    NaN  CFS  OLD_FLOW_GATE   FPV1MA
3 2015-12-31 01:00:00    NaN  CFS  OLD_FLOW_GATE   FPV1MA
4 2015-12-31 01:15:00    NaN  CFS  OLD_FLOW_GATE   FPV1MA


C:\Users\Inigo\AppData\Local\Temp\ipykernel_36968\635767342.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hydro_gateop.rename(columns={'MID_GATEOP':'MHO','GLC_GATEOP':'DGL','OLD_GATEOP':'OLD'}, inplace=True) # renaming to make it easier later.


In [99]:
gatef = {'name' : ['GrantLine','MiddleRiver','OldRiver'],
         'width' : [5,5,5],
         'C' : [0.8,0.8,0.8],
         'bottom_elev' :[-6,-5,-7],
         'ID': ['GLC','MID','OLD'],
         'station' : ['DGL','MHO','OLD'],
         "flow_op" : ["GLC_FLOW_FISH", "MID_FLOW_FISH", "OLD_FLOW_FISH"],
         "gate_status" : ["GLC_GATE_UP", "MID_GATE_UP", "OLD_GATE_UP"]
         }

full_data = {}

for i, gate_name in enumerate(gatef['name']):

    flow_data = sdg_flow[sdg_flow.gate_op == gatef["flow_op"][i]]
    gate_data = sdg_stage[sdg_stage.gate_op == gatef['gate_status'][i]]
    
    # Drop rows with NaN values from flow_data and gate_data
    flow_data = flow_data.dropna()
    gate_data = gate_data.dropna()
    
    full_data[gatef["ID"][i]] = {
        'name': gatef['name'][i],
        'bottom_elev': gatef['bottom_elev'][i],
        'width': gatef['width'][i],
        'flow_data': flow_data,
        'gate_data': gate_data
    }
sdg_flow_GLC_FLOW_FISH = full_data['GLC']['flow_data']['value']
sdg_flow_GLC_GATE_UP = full_data['GLC']['gate_data']['value']
glc_bottom_elev = full_data['GLC']['bottom_elev']
glc_width = full_data['GLC']['width']

In [92]:
print(sdg_flow_GLC_FLOW_FISH.dtypes)  # To check data types


float64


In [102]:

def calc_vel(flow,stage_up, bottom_elev, width):
    #velocity is flow/cross-section
    
    xs = (stage_up-bottom_elev)*width
    vel = flow/xs
    return vel

thr_vel = 8
# mask = (sdg_flow.index.month>=5) & (sdg_flow.index.month<12) # time when gate are in operation
# vel=pd.DataFrame()
# print(sdg_flow['scenario'].unique())
full_data["GLC"]['vel'] = calc_vel(sdg_flow_GLC_FLOW_FISH,sdg_flow_GLC_GATE_UP,glc_bottom_elev, glc_width)
# vel['MID'] = calc_vel(sdg_flow.MID_FLOW_FISH,sdg_flow.MID_GATE_UP,gatef['bottom_elev'][1], gatef['width'][1] )
# vel['OLD'] = calc_vel(sdg_flow.OLD_FLOW_FISH,sdg_flow.OLD_GATE_UP,gatef['bottom_elev'][2], gatef['width'][2] )
print(full_data["GLC"]['vel'].tail())
print(full_data['GLC'].keys())

63453    13.879196
63454     0.199047
63454    14.140911
63455     0.199117
63455    14.409926
Name: value, dtype: float64
dict_keys(['name', 'bottom_elev', 'width', 'flow_data', 'gate_data', 'vel'])
